# Gene analysis

In [2]:
import pandas as pd
import numpy as np

In [44]:
def write_answer(task, ans):
    with open("bio_" + str(task)+".txt", "w") as fout:
        fout.write(str(ans))

In [4]:
data = pd.read_csv("gene.csv")
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [17]:
print(data.Diagnosis.unique())
data_normal = data[data.Diagnosis == 'normal']
data_early = data[data.Diagnosis == 'early neoplasia']
data_cancer = data[data.Diagnosis == 'cancer']

['normal' 'early neoplasia' 'cancer']


In [58]:
from scipy.stats import ttest_ind
from more_itertools import ilen

pvals1 = [ttest_ind(data_normal[gene], data_early[gene], equal_var=False).pvalue for gene in data.columns[2:]]
pvals2 = [ttest_ind(data_early[gene], data_cancer[gene], equal_var=False).pvalue for gene in data.columns[2:]]
count1 = ilen(filter(lambda x: x < 0.05, pvals1))
count2 = ilen(filter(lambda x: x < 0.05, pvals2))

In [59]:
print(count1)
print(count2)

1575
3490


In [49]:
write_answer(1, count1)
write_answer(2, count2)

In [105]:
def fold_change(control, treatment):
    C = control.values.mean()
    T = treatment.values.mean()
    if (T > C):
        return T / C
    else:
        return -C / T

In [110]:
from statsmodels.stats import multitest

rejected1, p_corr1, _, _ = multitest.multipletests(pvals1, alpha=0.025, method='holm')
rejected2, p_corr2, _, _ = multitest.multipletests(pvals2, alpha=0.025, method='holm')

indexes1 = np.argwhere(rejected1 == True)
indexes2 = np.argwhere(rejected2 == True)
    
count = 0
for index in indexes1:
    FC = fold_change(data_normal.iloc[:,index + 2], data_early.iloc[:, index + 2])
    if (abs(FC) > 1.5):
        count+=1

write_answer(3, count)

count = 0
for index in indexes2:
    FC = fold_change(data_early.iloc[:,index + 2], data_cancer.iloc[:, index + 2])
    if (abs(FC) > 1.5):
        count+=1
        
print(count)

write_answer(4, count)

77


In [112]:
rejected1, p_corr1, _, _ = multitest.multipletests(pvals1, alpha=0.025, method='fdr_bh')
rejected2, p_corr2, _, _ = multitest.multipletests(pvals2, alpha=0.025, method='fdr_bh')

indexes1 = np.argwhere(rejected1 == True)
indexes2 = np.argwhere(rejected2 == True)
    
count = 0
for index in indexes1:
    FC = fold_change(data_normal.iloc[:,index + 2], data_early.iloc[:, index + 2])
    if (abs(FC) > 1.5):
        count+=1

write_answer(5, count)

count = 0
for index in indexes2:
    FC = fold_change(data_early.iloc[:,index + 2], data_cancer.iloc[:, index + 2])
    if (abs(FC) > 1.5):
        count+=1
        
print(count)

write_answer(6, count)

524


# Credit scoring

In [123]:
credit_data = pd.read_csv("credit_card.csv")
returned_data = creadit_data[credit_data.default == 0]
not_returned_data = creadit_data[credit_data.default == 1]